In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def train_loop(dataloader, model, loss_fn, optimizer, train_loss_history):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        l2_lambda = 0.001
        l2_norm = sum(p.pow(2.0).sum()
                  for p in model.parameters())

        loss = loss + l2_lambda * l2_norm


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 2000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    train_loss_history.append(loss.item())
    return train_loss_history


def test_loop(dataloader, model, loss_fn, loss_history, accuracy_history):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    loss_history.append(test_loss)
    correct /= size
    accuracy_history.append(100*correct)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return loss_history, accuracy_history

In [ ]:
learning_rates = [0.001]
batch_sizes = [16]
for i in range(len(learning_rates)): 

  for j in range(len(batch_sizes)):
    batch_size = batch_sizes[j]
    learning_rate = learning_rates[i]

    print("----------Learning Rate = ", learning_rate)
    print("----------Batch Size    = ", batch_size)

    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.RandomHorizontalFlip(0.5)])
    
    augment_transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.RandomHorizontalFlip(1)])

    augmented_dataset1 = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=augment_transform)
    augmented_dataset2 = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=transform)

    train_dataset = torch.utils.data.ConcatDataset([augmented_dataset1, augmented_dataset2])


    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    class Net(nn.Module):
      def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Conv2d(3,8,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(8,16,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(16),

            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128,256,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(256),

            nn.Flatten(),
            nn.Linear(4096, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

      def forward(self, x):
          x = self.stack(x)
          return x  

model = Net()
from torchsummary import summary
summary(model,(3,32,32))

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum= 0.9)

import time as time


epoch = []
test_loss_history = []
train_loss_history = []
accuracy_history = []

epochs = 10

start = time.time()

for t in range(epochs):
    print("Epoch" , t+1, "-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer, train_loss_history)
    test_loop(test_loader, model, loss_fn, test_loss_history,accuracy_history)
    epoch.append(t)

finish = time.time()
total = finish - start
total_m, total_s = divmod(total, 60)

print(total_m, "m", round(total_s,4), "s")
print("Done!")

----------Learning Rate =  0.001
----------Batch Size    =  16


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             224
              ReLU-2            [-1, 8, 32, 32]               0
            Conv2d-3           [-1, 16, 32, 32]           1,168
              ReLU-4           [-1, 16, 32, 32]               0
         MaxPool2d-5           [-1, 16, 16, 16]               0
       BatchNorm2d-6           [-1, 16, 16, 16]              32
            Conv2d-7           [-1, 32, 16, 16]           4,640
              ReLU-8           [-1, 32, 16, 16]               0
            Conv2d-9           [-1, 64, 16, 16]          18,496
             ReLU-10           [-1, 64, 16, 16]               0
        MaxPool2d-11             [-1, 64, 8, 8]               0
      BatchNorm2d-12             [-1, 64, 8,

In [ ]:
def plot_loss(history, history2, epoch):
    
    plt.figure(1)
    train_accuracies = [x for x in history]
    test_accuracies = [y for y in history2]
    plt.plot(epoch, train_accuracies, label = "train loss")
    plt.plot(epoch, test_accuracies, label = "test loss")

    plt.xlabel('epoch')
    plt.ylabel('Loss')
    plt.title('Loss vs. No. of epochs');
    plt.locator_params('x', nbins=len(history))
    plt.locator_params('x', nbins=len(history2))

    plt.legend()

def plot_accuracy(history, epoch):
  plt.figure(2)
  accuracies = [x for x in history]
  plt.plot(epoch, accuracies, label = "accuracy")

  plt.xlabel("epoch")
  plt.ylabel("accuracy")
  plt.title("Accuracy vs. No. of epochs")
  plt.locator_params('x',nbins=len(history))
  

plot_loss(test_loss_history , train_loss_history, epoch)
plot_accuracy(accuracy_history, epoch)